In [1]:
import re
import urllib
import os
import time
import pandas as pd
import string
import nltk

from selenium import webdriver
from nltk.sentiment import sentiment_analyzer
from nltk.sentiment import SentimentIntensityAnalyzer
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
driver4 = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [4]:
url4="https://www.daraz.pk/"

In [5]:
driver4.get(url4)

In [6]:
searchbar4=driver4.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
searchbar4.send_keys('mobile phones' + Keys.RETURN)

## Task 1

In [ ]:
product_id=[]
product_name = []
product_price = []
product_description=[]
product_brand=[]
product_indivisual_reviews=[]
product_combine_reviews=[]
product_score=[]
product_sentiment=[]
product_review_id=[]
product_rating=[]
product_links=[]
score=0
countofreveiw=0
total_product=0

next_button = driver4.find_element(By.CLASS_NAME, "ant-pagination-next")
for page_num in range(3):
    print('Page number',page_num)
    products_grid = driver4.find_element(By.CLASS_NAME, "box--ujueT")
    products_list = products_grid.find_elements(By.CLASS_NAME, "gridItem--Yd0sa")

    for index in range(len(products_list)-1):
        
        try:
            print('Index',index)
            products_grid = driver4.find_element(By.CLASS_NAME, "box--ujueT")
            products_list = products_grid.find_elements(By.CLASS_NAME, "gridItem--Yd0sa")
            i = products_grid.find_elements(By.CLASS_NAME, "gridItem--Yd0sa")[index]
            
            #product name
            nameclass = i.find_element(By.CLASS_NAME, "title--wFj93").text
            print("name:", nameclass)
        
            #checking condition for mobile phone covers and excluding them
            if("case" in nameclass or "cover"in nameclass or "charger"in nameclass):
                print("case error")
                continue
        
            product_name.append(nameclass)

            #product price
            priceclass = i.find_element(By.CLASS_NAME, "price--NVB62").text
            product_price.append(priceclass)
            print("price: ",priceclass)
            
            
            #product link
            linkclass = i.find_element(By.ID, "id-a-link")
            product_link = linkclass.get_attribute('href')
            print("Link: ",product_link)
            product_links.append(product_link)
            
            driver4.get(product_link)
            time.sleep(2)
            
            
            #brand name
            try:
                brand_class=driver4.find_element(By.CLASS_NAME,"pdp-product-brand__brand-link").text
                product_brand.append(brand_class)
                print('Brand: ',brand_class)
                
            except NoSuchElementException:
                print("No brand")
                product_brand.append("No brand")

            #creating unique id
            unique_id = int(time.time())
            product_id.append(unique_id)

                
            #scroll to product description to load them
            try:
                scrolling = driver4.find_element(By.ID, "module_product_detail")
                driver4.execute_script("arguments[0].scrollIntoView(true);", scrolling)
                time.sleep(2)

            except NoSuchElementException:
                print("Error in scrolling")
            
            #description storing
            try:
                description_class=driver4.find_element(By.CLASS_NAME,'html-content.pdp-product-highlights').text
                product_description.append(description_class)
                print("Description")
            
            except NoSuchElementException:
                print("No description")
                product_description.append("No description")
            
            #scroll to get reviews
            try:
                element = driver4.find_element(By.ID, "block-pjLIzlmQY9f")
                driver4.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)
            
            except NoSuchElementException:
                print("Error in scrolling reviews")
            
            
            #creating unique id
            review_unique_id = int(time.time())
            product_review_id.append(review_unique_id)
            
            #getting the box of reviews
            
            try:
                prodrattingclass = driver4.find_element(By.ID, "module_product_review")
                
                try:
                    #product reviews
                    productrevclass = prodrattingclass.find_element(By.CLASS_NAME, "score")
                    product_rating.append(productrevclass.text)
                except NoSuchElementException:
                    product_rating.append("No rating")
            

                try:
                    #review text
                    reviews_Section=prodrattingclass.find_element(By.CLASS_NAME,'review-content')
                    reviews_indivisual=reviews_Section.find_elements(By.CLASS_NAME,"review-content-sl")

                    for reviewsiter in reviews_indivisual:

                        #when review is empty
                        if(reviewsiter.text==''):
                            continue
                        countofreveiw+=1

                        strforreview=reviewsiter.text
                        product_indivisual_reviews.append(strforreview)

                        sid=SentimentIntensityAnalyzer()

                        intensity=sid.polarity_scores(strforreview)

                        if intensity["compound"]>=0.05:
                            product_reviews_sentiments="Positive"
                        elif intensity["compound"]<=-0.05:
                            product_reviews_sentiments="Negative"
                        else:
                            product_reviews_sentiments="Neutral"

                        product_sentiment.append(product_reviews_sentiments) 

                    print(product_sentiment)
                    score_per_review = {
                        'Positive': 100,
                        'Negative': 0,
                        'Neutral':50
                    }

                    # Calculate the total score
                    total_score = sum(score_per_review[review] for review in product_sentiment)

                    # Calculate the average score
                    print("total_score: ",total_score," len(reviews_list: ",len(product_sentiment))
                    average_score = int(total_score / len(product_sentiment))

                    print(f"The customer's score is {average_score}")

                    product_combine_reviews.append(product_indivisual_reviews)
                    product_score.append(average_score)

                    product_indivisual_reviews=[]
                    product_sentiment=[]
                    total_score=0

                #when no review is entered
                except NoSuchElementException:
                    print("No review")
                    product_combine_reviews.append("No review")
                    product_score.append(0)

            
            except NoSuchElementException:
                product_combine_reviews.append("No review")
                product_score.append(0)
                product_rating.append("No rating")
                print("No review")
            
            driver4.execute_script("window.history.go(-1)")
            time.sleep(3)
        
        
        except StaleElementReferenceException:
            products_grid = driver4.find_element(By.CLASS_NAME, "box--ujueT")
            products_list = products_grid.find_elements(By.CLASS_NAME, "gridItem--Yd0sa")

    try:
        next_button = driver4.find_element(By.CLASS_NAME, "ant-pagination-next")
        next_button.click()
        time.sleep(5)
    except NoSuchElementException:
        print("No more pages available.")
        break

    

In [57]:
# Create a DataFrame
df = pd.DataFrame({
    'Product ID': original_product_id,
    'Product Name': original_product_name,
    'Product Price': original_product_price,
    'Product Brand': orignal_product_brand,
    'Product Description': original_product_description,
    'Product link': orignal_product_link
})


In [3]:
import pandas as pd

In [45]:
df.to_csv('output.csv', mode='a', header=False, index=False)

In [58]:
df_review=pd.DataFrame({
    'Review_ID': original_product_id,
    'Product rating':orignal_product_rating,
    'Product reviews': original_product_combine_reviews,
    'Review score': original_product_score
})

In [47]:
df_review.to_csv('review.csv', mode='a', header=False, index=False)

In [4]:
df_details=pd.read_csv("output.csv")
df_details.set_index("Product ID", inplace=True)
df_details

,Product Name,Product Price,Product Brand,Product Description,Product link
Product ID,,,,,
1701508000,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/infinix-note-30-...
1701508015,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 64,999",Xiaomi,Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...,https://www.daraz.pk/products/a60s-wach-4gb-12...
1701508035,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",No Brand,Release Date2023-05-22SIM SupportDual SIM (Nan...,https://www.daraz.pk/products/a2-3gb-64gb-8mp-...
1701508055,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",No Brand,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/redmi-note-12s-6...
1701508072,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 97,999",Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/s23-8-128-5000ma...
...,...,...,...,...,...
1701515539,"Xiaomi Redmi 12 , 4GB RAM 128GB ROM , 50 MP Ma...","Rs. 38,999",No Brand,BuildOSAndroid 13 OS UIMIUI 14 Dimensions168.6...,https://www.daraz.pk/products/12-4gb-128gb-50-...
1701515559,"Infinix Note 30 Pro , 8GB RAM 256GB ROM , 108M...","Rs. 65,999",No Brand,General FeaturesRelease Date2023-05-22SIM Supp...,https://www.daraz.pk/products/30-8gb-256gb-108...
1701515580,Redmi 12C 4GB RAM + 128GB ROM PTA Approved Mob...,"Rs. 30,940",No Brand,Redmi 12C\nRAM: 4GB\nROM: 128GB\nOfficial Bran...,https://www.daraz.pk/products/4gb-12c-128gb-pt...


In [5]:
df_details['Product Price'] = df_details['Product Price'].replace('[^\d]', '', regex=True).astype(int)
df_details

,Product Name,Product Price,Product Brand,Product Description,Product link
Product ID,,,,,
1701508000,Like New Phones - Used Apple iPhone X - Silver...,106999,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/infinix-note-30-...
1701508015,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,64999,Xiaomi,Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...,https://www.daraz.pk/products/a60s-wach-4gb-12...
1701508035,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,No Brand,Release Date2023-05-22SIM SupportDual SIM (Nan...,https://www.daraz.pk/products/a2-3gb-64gb-8mp-...
1701508055,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,No Brand,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/redmi-note-12s-6...
1701508072,Like New Phones - Used Apple iPhone X - Silver...,97999,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/s23-8-128-5000ma...
...,...,...,...,...,...
1701515539,"Xiaomi Redmi 12 , 4GB RAM 128GB ROM , 50 MP Ma...",38999,No Brand,BuildOSAndroid 13 OS UIMIUI 14 Dimensions168.6...,https://www.daraz.pk/products/12-4gb-128gb-50-...
1701515559,"Infinix Note 30 Pro , 8GB RAM 256GB ROM , 108M...",65999,No Brand,General FeaturesRelease Date2023-05-22SIM Supp...,https://www.daraz.pk/products/30-8gb-256gb-108...
1701515580,Redmi 12C 4GB RAM + 128GB ROM PTA Approved Mob...,30940,No Brand,Redmi 12C\nRAM: 4GB\nROM: 128GB\nOfficial Bran...,https://www.daraz.pk/products/4gb-12c-128gb-pt...


In [6]:
df_reviews=pd.read_csv("review.csv")
df_reviews.set_index("Review_ID", inplace=True)
df_reviews

,Product rating,Product reviews,Review score
Review_ID,,,
1701508000,5,['Received as promised almost 10/10 condition ...,100
1701508015,4.7,['this is the initial review. the phone is the...,100
1701508035,4.7,"['I got what was shown, but I want to tell abo...",100
1701508055,4.1,['#11.11 PTA approved device received safely. ...,100
1701508072,5,['highly recommend...'],50
...,...,...,...
1701515539,5,['I would like to appreciate Daraz and the sel...,100
1701515559,3,"[""fraud seller. sent lower spec phone but show...",0
1701515580,5,"[""Alhamdulillah, With in 3 day's I have receiv...",100


In [7]:
df_reviews['Product rating']=df_reviews['Product rating'].str.replace('No rating','0')
df_reviews['Product rating']=df_reviews['Product rating'].astype(float)

In [8]:
df_details

,Product Name,Product Price,Product Brand,Product Description,Product link
Product ID,,,,,
1701508000,Like New Phones - Used Apple iPhone X - Silver...,106999,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/infinix-note-30-...
1701508015,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,64999,Xiaomi,Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...,https://www.daraz.pk/products/a60s-wach-4gb-12...
1701508035,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,No Brand,Release Date2023-05-22SIM SupportDual SIM (Nan...,https://www.daraz.pk/products/a2-3gb-64gb-8mp-...
1701508055,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,No Brand,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/redmi-note-12s-6...
1701508072,Like New Phones - Used Apple iPhone X - Silver...,97999,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/s23-8-128-5000ma...
...,...,...,...,...,...
1701515539,"Xiaomi Redmi 12 , 4GB RAM 128GB ROM , 50 MP Ma...",38999,No Brand,BuildOSAndroid 13 OS UIMIUI 14 Dimensions168.6...,https://www.daraz.pk/products/12-4gb-128gb-50-...
1701515559,"Infinix Note 30 Pro , 8GB RAM 256GB ROM , 108M...",65999,No Brand,General FeaturesRelease Date2023-05-22SIM Supp...,https://www.daraz.pk/products/30-8gb-256gb-108...
1701515580,Redmi 12C 4GB RAM + 128GB ROM PTA Approved Mob...,30940,No Brand,Redmi 12C\nRAM: 4GB\nROM: 128GB\nOfficial Bran...,https://www.daraz.pk/products/4gb-12c-128gb-pt...


In [9]:
df_reviews

,Product rating,Product reviews,Review score
Review_ID,,,
1701508000,5.0,['Received as promised almost 10/10 condition ...,100
1701508015,4.7,['this is the initial review. the phone is the...,100
1701508035,4.7,"['I got what was shown, but I want to tell abo...",100
1701508055,4.1,['#11.11 PTA approved device received safely. ...,100
1701508072,5.0,['highly recommend...'],50
...,...,...,...
1701515539,5.0,['I would like to appreciate Daraz and the sel...,100
1701515559,3.0,"[""fraud seller. sent lower spec phone but show...",0
1701515580,5.0,"[""Alhamdulillah, With in 3 day's I have receiv...",100


In [12]:
df_details.to_csv("output.csv")

In [13]:
df_reviews.to_csv("review.csv")

## Task 2

In [106]:
import sqlite3

In [107]:
connection=sqlite3.connect("detailsofphone.db")

In [108]:
cursor=connection.cursor()

In [83]:
#INTEGER
#REAL(FLOAT)
#TEXT
#BLOBS (BINARY LARGE OBJECTS)
#NULL


In [84]:
cursor.execute('CREATE TABLE Products(ID INTEGER, Name TEXT,Price INT, Brand TEXT, Specification TEXT)')

In [85]:
connection.commit()

In [109]:
import sqlalchemy

In [91]:
engine=sqlalchemy.create_engine('sqlite:///detailsofphone.db')

In [92]:
df_details.to_sql('Product',engine)

184

In [98]:
df_reviews.to_sql('Reviews',engine)

184

In [115]:
cursor.execute("SELECT * FROM Product WHERE `Product Price` BETWEEN 30000 AND 40000")
result = cursor.fetchall()

result=pd.DataFrame(result)
result

,0,1,2,3,4,5
0,1701508055,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,No Brand,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/redmi-note-12s-6...
1,1701508321,"Infinix Hot 30 , 8GB RAM 128GB ROM , 50MP Main...",36999,No Brand,General FeaturesRelease Date2023-08-29SIM Supp...,https://www.daraz.pk/products/oppo-a57-2022-65...
2,1701508352,Redmi 12C 4GB RAM + 128GB ROM PTA Approved Mob...,33999,Xiaomi,Redmi 12C RAM: 4GB ROM: 128GB Official Brand S...,https://www.daraz.pk/products/12-8gb-128gb-667...
3,1701508486,Infinix Hot 30 || 8GB+8GB Ram 128GB Rom || 500...,36999,No Brand,General FeaturesRelease Date2023-08-29SIM Supp...,https://www.daraz.pk/products/samsung-a54-5g-6...
4,1701508596,Samsung Galaxy A04s 4GB - 128GB OFFICIAL PTA A...,39999,No Brand,Official PTA Approved\n1 Year official Warrant...,https://www.daraz.pk/products/s23-wace-8gb-8gb...
5,1701508774,"Infinix Hot 30 8GB/128GB - Android 13 - 6.78"" ...",37999,No Brand,"• Infinix Hot 30 with 6.78"" display.• 5000 mAh...",https://www.daraz.pk/products/30-4-gb-64-gb-8-...
6,1701508901,Itel S23 - 8GB Ram - 256GB Rom - 5000mAh Battery,32500,No Brand,BODY\nDimensions\n164 x 76 x 8 mm (6.46 x 2.99...,https://www.daraz.pk/products/s23-8-128-5000ma...
7,1701509121,Redmi 12 || 4GB Ram 128GB Rom || 5000mAh Battery,36999,No Brand,General FeaturesRelease Date2023-09-12SIM Supp...,https://www.daraz.pk/products/12-4gb-1-000-gb-...
8,1701509196,itel S23 6.6″ HD+ Display | Up to 16+256GB Sto...,33499,itel,"6.6"" HD+ IPS Display\n16GB* Customizable RAM w...",https://www.daraz.pk/products/s23-66-hd-16-256...
9,1701509255,"Tecno Spark 10 Pro , 8GB RAM 128GB ROM , Back ...",37999,No Brand,6.78 InchesDisplay\n8 GBRAM\n5000 mAhBattery\n...,https://www.daraz.pk/products/10-8gb-128gb-50-...


In [116]:
cursor.execute("SELECT AVG(`Product rating`) FROM Reviews")
average_rating = cursor.fetchone()[0]

print("Average Rating:", average_rating)


Average Rating: 4.046195652173912


## Task 3

In [14]:
import sqlite3
import sqlalchemy
connection=sqlite3.connect("detailsofphone.db")
cursor=connection.cursor()

engine=sqlalchemy.create_engine('sqlite:///detailsofphone.db')

cursor.execute("SELECT * FROM Product WHERE `Product Price` BETWEEN 30000 AND 40000")
result = cursor.fetchall()

result=pd.DataFrame(result)
result

In [15]:
connection=sqlite3.connect("detailsofphone.db")

In [16]:
cursor=connection.cursor()

In [67]:
query = f"""
    SELECT P.`Product Name`, P.`Product Price`, R.`Product Rating`, R.`Review Score`, P.`Product link`
    FROM Product P
    JOIN Reviews R ON P.`Product ID` = R.`Review_ID` 
    ORDER BY R.`Product Rating` DESC, R.`Review Score` DESC
    LIMIT 5
"""

result = execute_sql_query(query)


listofdict=[]
for i in result:
    mydict={}
    mydict["name"]=i[0]
    mydict["price"]=i[1]
    mydict["rating"]=i[2]
    mydict["score"]=i[3]
    mydict["link"]=i[4]
    listofdict.append(mydict)


[{'name': 'Like New Phones - Used Apple iPhone X - Silver 256 GB - PTA Approved',
  'price': 106999,
  'rating': 5.0,
  'score': 100,
  'link': 'https://www.daraz.pk/products/infinix-note-30-ram-8gb8gb-256gb-storage-678-fhd-5000mah-mega-battery-pta-approved-official-warranty-i429046135-s2098320869.html?search=1'},
 {'name': 'Like New Phones - Used Apple iPhone SE 2nd Gen (2020) - White 64 GB - PTA Approved',
  'price': 70999,
  'rating': 5.0,
  'score': 100,
  'link': 'https://www.daraz.pk/products/20-8gb-26gb-64-g99-1-mah-21-i433969037-s2095364064.html?search=1'},
 {'name': 'Like New Phones - Used Apple iPhone SE 2nd Gen (2020) - Black 64 GB - PTA Approved',
  'price': 70999,
  'rating': 5.0,
  'score': 100,
  'link': 'https://www.daraz.pk/products/like-new-phones-used-apple-iphone-8-plus-gold-64-gb-pta-approved-i434302391-s2082871184.html?search=1'},
 {'name': 'Like New Phones - Used Apple iPhone SE 2nd Gen (2020) - Red 64 GB - PTA Approved',
  'price': 70999,
  'rating': 5.0,
  'sco